AntiFraud
============





A classifier that can detect with a high level of accuracy whether a given credit card transaction is legitimate or fraudulent. 

The dataset has been collected and analyzed during a research collaboration of Worldline and the Machine Learning Group of ULB (Université Libre de Bruxelles) on big data mining and fraud detection. The datasets contain transactions made using credit cards in September 2013 by European cardholders. The dataset contains transactions that occurred over the period of two days. The dataset is highly unbalanced: 492 frauds out of 284,807 transactions, so the fraudulent transaction class account for 0.172% of all payments.

I addressed the data imbalance by performing randomized sampling of the overrepresented class to reduce it to the size of 2x of the fraudlent class size. I have created a CNN model using Conv1D layers (RELU), Dense layers, and Batch Normalization. My goal was to achieve high accuracy. My current model yields over 98% accuracy rate after 30 epochs.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('creditcard.csv')

In [3]:
df

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


In [4]:
df.Class.value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [5]:
fraud = df[df.Class==1]
legit = df[df.Class==0]

In [6]:
fraud

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
541,406.0,-2.312227,1.951992,-1.609851,3.997906,-0.522188,-1.426545,-2.537387,1.391657,-2.770089,...,0.517232,-0.035049,-0.465211,0.320198,0.044519,0.177840,0.261145,-0.143276,0.00,1
623,472.0,-3.043541,-3.157307,1.088463,2.288644,1.359805,-1.064823,0.325574,-0.067794,-0.270953,...,0.661696,0.435477,1.375966,-0.293803,0.279798,-0.145362,-0.252773,0.035764,529.00,1
4920,4462.0,-2.303350,1.759247,-0.359745,2.330243,-0.821628,-0.075788,0.562320,-0.399147,-0.238253,...,-0.294166,-0.932391,0.172726,-0.087330,-0.156114,-0.542628,0.039566,-0.153029,239.93,1
6108,6986.0,-4.397974,1.358367,-2.592844,2.679787,-1.128131,-1.706536,-3.496197,-0.248778,-0.247768,...,0.573574,0.176968,-0.436207,-0.053502,0.252405,-0.657488,-0.827136,0.849573,59.00,1
6329,7519.0,1.234235,3.019740,-4.304597,4.732795,3.624201,-1.357746,1.713445,-0.496358,-1.282858,...,-0.379068,-0.704181,-0.656805,-1.632653,1.488901,0.566797,-0.010016,0.146793,1.00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279863,169142.0,-1.927883,1.125653,-4.518331,1.749293,-1.566487,-2.010494,-0.882850,0.697211,-2.064945,...,0.778584,-0.319189,0.639419,-0.294885,0.537503,0.788395,0.292680,0.147968,390.00,1
280143,169347.0,1.378559,1.289381,-5.004247,1.411850,0.442581,-1.326536,-1.413170,0.248525,-1.127396,...,0.370612,0.028234,-0.145640,-0.081049,0.521875,0.739467,0.389152,0.186637,0.76,1
280149,169351.0,-0.676143,1.126366,-2.213700,0.468308,-1.120541,-0.003346,-2.234739,1.210158,-0.652250,...,0.751826,0.834108,0.190944,0.032070,-0.739695,0.471111,0.385107,0.194361,77.89,1
281144,169966.0,-3.113832,0.585864,-5.399730,1.817092,-0.840618,-2.943548,-2.208002,1.058733,-1.632333,...,0.583276,-0.269209,-0.456108,-0.183659,-0.328168,0.606116,0.884876,-0.253700,245.00,1


In [7]:
# get the size of the fraudlent class
fraudSize = len(fraud.index)

In [8]:
# randomly sample legitemate transactions to make up 2/3 of the total dataset
# Seed the RNG to 1337 for reproducibility
notfraud = legit.sample(fraudSize*2, random_state=1337)

In [9]:
len(notfraud.index)

984

In [10]:
# udf -- undersampled data frame (where imbalance is addressed by random sampling of the overrepresented class)

udf = fraud.append(notfraud, ignore_index=True)

In [11]:
udf

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,406.0,-2.312227,1.951992,-1.609851,3.997906,-0.522188,-1.426545,-2.537387,1.391657,-2.770089,...,0.517232,-0.035049,-0.465211,0.320198,0.044519,0.177840,0.261145,-0.143276,0.00,1
1,472.0,-3.043541,-3.157307,1.088463,2.288644,1.359805,-1.064823,0.325574,-0.067794,-0.270953,...,0.661696,0.435477,1.375966,-0.293803,0.279798,-0.145362,-0.252773,0.035764,529.00,1
2,4462.0,-2.303350,1.759247,-0.359745,2.330243,-0.821628,-0.075788,0.562320,-0.399147,-0.238253,...,-0.294166,-0.932391,0.172726,-0.087330,-0.156114,-0.542628,0.039566,-0.153029,239.93,1
3,6986.0,-4.397974,1.358367,-2.592844,2.679787,-1.128131,-1.706536,-3.496197,-0.248778,-0.247768,...,0.573574,0.176968,-0.436207,-0.053502,0.252405,-0.657488,-0.827136,0.849573,59.00,1
4,7519.0,1.234235,3.019740,-4.304597,4.732795,3.624201,-1.357746,1.713445,-0.496358,-1.282858,...,-0.379068,-0.704181,-0.656805,-1.632653,1.488901,0.566797,-0.010016,0.146793,1.00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1471,156987.0,-0.084911,0.635563,0.898207,-0.751652,0.194008,0.198208,0.217612,0.138188,0.356172,...,-0.016602,-0.094023,0.147583,0.673185,-0.674288,-0.774377,-0.194986,0.014313,5.99,0
1472,70760.0,1.210386,0.043092,-0.310696,-0.069208,0.124064,-0.782460,0.505576,-0.369676,-0.244961,...,-0.434789,-1.404629,0.018536,-0.402937,0.233870,0.661375,-0.106990,0.011884,84.96,0
1473,41165.0,-0.467546,0.538498,0.400995,0.730403,0.163646,1.883606,1.318606,0.173549,-0.401738,...,-0.104982,-0.141266,-0.259354,-1.699297,0.488462,-0.146819,-0.004959,-0.081395,228.00,0
1474,153598.0,-0.078001,0.993182,-1.882856,-0.451481,0.866597,-0.967565,1.277695,0.180038,-0.800779,...,0.287604,0.614922,0.106451,0.706389,-0.395661,0.488815,-0.123975,0.034691,85.25,0


In [12]:
y = udf['Class']
X = udf.drop(['Class'],axis=1)

In [13]:
y


0       1
1       1
2       1
3       1
4       1
       ..
1471    0
1472    0
1473    0
1474    0
1475    0
Name: Class, Length: 1476, dtype: int64

In [14]:
X

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,406.0,-2.312227,1.951992,-1.609851,3.997906,-0.522188,-1.426545,-2.537387,1.391657,-2.770089,...,0.126911,0.517232,-0.035049,-0.465211,0.320198,0.044519,0.177840,0.261145,-0.143276,0.00
1,472.0,-3.043541,-3.157307,1.088463,2.288644,1.359805,-1.064823,0.325574,-0.067794,-0.270953,...,2.102339,0.661696,0.435477,1.375966,-0.293803,0.279798,-0.145362,-0.252773,0.035764,529.00
2,4462.0,-2.303350,1.759247,-0.359745,2.330243,-0.821628,-0.075788,0.562320,-0.399147,-0.238253,...,-0.430022,-0.294166,-0.932391,0.172726,-0.087330,-0.156114,-0.542628,0.039566,-0.153029,239.93
3,6986.0,-4.397974,1.358367,-2.592844,2.679787,-1.128131,-1.706536,-3.496197,-0.248778,-0.247768,...,-0.171608,0.573574,0.176968,-0.436207,-0.053502,0.252405,-0.657488,-0.827136,0.849573,59.00
4,7519.0,1.234235,3.019740,-4.304597,4.732795,3.624201,-1.357746,1.713445,-0.496358,-1.282858,...,0.009061,-0.379068,-0.704181,-0.656805,-1.632653,1.488901,0.566797,-0.010016,0.146793,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1471,156987.0,-0.084911,0.635563,0.898207,-0.751652,0.194008,0.198208,0.217612,0.138188,0.356172,...,-0.145993,-0.016602,-0.094023,0.147583,0.673185,-0.674288,-0.774377,-0.194986,0.014313,5.99
1472,70760.0,1.210386,0.043092,-0.310696,-0.069208,0.124064,-0.782460,0.505576,-0.369676,-0.244961,...,0.205985,-0.434789,-1.404629,0.018536,-0.402937,0.233870,0.661375,-0.106990,0.011884,84.96
1473,41165.0,-0.467546,0.538498,0.400995,0.730403,0.163646,1.883606,1.318606,0.173549,-0.401738,...,0.011129,-0.104982,-0.141266,-0.259354,-1.699297,0.488462,-0.146819,-0.004959,-0.081395,228.00
1474,153598.0,-0.078001,0.993182,-1.882856,-0.451481,0.866597,-0.967565,1.277695,0.180038,-0.800779,...,-0.152917,0.287604,0.614922,0.106451,0.706389,-0.395661,0.488815,-0.123975,0.034691,85.25


In [15]:
from sklearn.model_selection import train_test_split

# Test/train split:

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,stratify=y)

In [16]:
#Standardize features by removing the mean and scaling to unit variance


from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [17]:
# Convert to Numpy arrays

y_train=y_train.to_numpy()
y_test=y_test.to_numpy()

In [18]:
# CNN imports

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import BatchNormalization,Dropout,Dense,Flatten,Conv1D


In [19]:
X_train=X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_test=X_test.reshape(X_test.shape[0],X_test.shape[1],1)

In [20]:
# CNN architecture definition

model=Sequential(

    [
        Conv1D(64,2,activation='relu',input_shape=X_train[0].shape),
        BatchNormalization(), # Prevent internal covariate shift
        #Dropout(0.1),
        
        Conv1D(64,2,activation='relu'),
        BatchNormalization(),
        #Dropout(0.5),
        
        Flatten(),
        Dense(64,activation='relu'),
        #Dropout(0.5),
        
        Dense(1,activation='sigmoid')
    ]

)



In [21]:
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 29, 64)            192       
_________________________________________________________________
batch_normalization (BatchNo (None, 29, 64)            256       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 28, 64)            8256      
_________________________________________________________________
batch_normalization_1 (Batch (None, 28, 64)            256       
_________________________________________________________________
flatten (Flatten)            (None, 1792)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                114752    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 6

In [22]:
# Adaptive Learning Rate Gradient Descent
opt = Adam(learning_rate=0.0001)
# Binary crossentropy loss function for fraud/legit binary classification
model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])

In [23]:
timeline = model.fit(X_train,y_train,epochs=30,validation_data=(X_test,y_test))

Epoch 1/30
37/37 [==============================] - 0s 8ms/step - loss: 0.3777 - accuracy: 0.8288 - val_loss: 0.5774 - val_accuracy: 0.8750
Epoch 2/30
37/37 [==============================] - 0s 4ms/step - loss: 0.1733 - accuracy: 0.9441 - val_loss: 0.5311 - val_accuracy: 0.8851
Epoch 3/30
37/37 [==============================] - 0s 3ms/step - loss: 0.1448 - accuracy: 0.9542 - val_loss: 0.4850 - val_accuracy: 0.9155
Epoch 4/30
37/37 [==============================] - 0s 3ms/step - loss: 0.1306 - accuracy: 0.9559 - val_loss: 0.4347 - val_accuracy: 0.9291
Epoch 5/30
37/37 [==============================] - 0s 4ms/step - loss: 0.1208 - accuracy: 0.9593 - val_loss: 0.3868 - val_accuracy: 0.9426
Epoch 6/30
37/37 [==============================] - 0s 3ms/step - loss: 0.1115 - accuracy: 0.9619 - val_loss: 0.3415 - val_accuracy: 0.9459
Epoch 7/30
37/37 [==============================] - 0s 4ms/step - loss: 0.1051 - accuracy: 0.9636 - val_loss: 0.3040 - val_accuracy: 0.9493
Epoch 8/30
37/37 [==


Dataset:
    Andrea Dal Pozzolo, Olivier Caelen, Reid A. Johnson and Gianluca Bontempi. Calibrating 
Probability with Undersampling for Unbalanced Classification. In Symposium on 
Computational Intelligence and Data Mining (CIDM), IEEE, 2015
